Social Network Analysis - Part 3 - Example 7
(Community Detection)

In this exercise, we will analyze the "Karate Club" network (https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

We will use different libraries for 3 different community detection methods, namely:
1. Louvain (https://en.wikipedia.org/wiki/Louvain_method)
2. Girvan-Newman (https://en.wikipedia.org/wiki/Girvan%E2%80%93Newman_algorithm)
3. Hierarchical Clustering (https://en.wikipedia.org/wiki/Hierarchical_clustering)


- Ref (nice article): https://towardsdatascience.com/graph-algorithms-part-2-dce0b2734a1d
- Ref (Python workbook) https://github.com/maelfabien/Machine_Learning_Tutorials/blob/master/3-MachineLearning/GraphMining/Graph_Analysis.ipynb

In [ ]:
import numpy as np
import random

from IPython.display import Image
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import networkx as nx
from networkx.algorithms import community

In [ ]:
# Node2Vec
from node2vec import Node2Vec

%matplotlib inline
from sklearn import metrics
import os.path
import urllib
import tarfile

In [ ]:
'''
We'll use the "Karate Club" graph integrated in networkx.
It represents the relations of members of a Karate Club.

However, due to a lack a agreement of the founders of the club, 
the club has recently been splitted in two.

We'll try to illustrate this event with graphs.
'''

# Load a sample network "Karate"
G_karate = nx.karate_club_graph()

In [ ]:
# Let's draw the graph
pos = nx.spring_layout(G_karate)
nx.draw(G_karate, cmap = plt.get_cmap('rainbow'), with_labels=True, pos=pos)

In [ ]:
# Let's describe the graph

# Network size (# of nodes)
n = G_karate.number_of_nodes()
print("Size of graph: ", n)

In [ ]:
# Degree Centrality of each node
G_karate.degree()

In [ ]:
degree_sequence = list(G_karate.degree())
degree_sequence

In [ ]:
nb_nodes = n
nb_arr = len(G_karate.edges())

avg_degree = np.mean(np.array(degree_sequence)[:,1])
med_degree = np.median(np.array(degree_sequence)[:,1])

max_degree = max(np.array(degree_sequence)[:,1])
min_degree = np.min(np.array(degree_sequence)[:,1])



print("Number of nodes : " + str(nb_nodes))
print("Number of edges : " + str(nb_arr))

print("Maximum degree : " + str(max_degree))
print("Minimum degree : " + str(min_degree))

print("Average degree : " + str(avg_degree))
print("Median degree : " + str(med_degree))

In [ ]:
# More visually, we can plot the histogram of the sequence of degrees
degree_freq = np.array(nx.degree_histogram(G_karate)).astype('float')

plt.figure(figsize=(12, 8))
plt.stem(degree_freq)
plt.ylabel("Frequence")
plt.xlabel("Degre")
plt.show()

In [ ]:
# Node-level clustering coefficient
# Local Clustering Coefficients
list(nx.clustering(G_karate).values())

# Clustering Coefficient
In graph theory, a clustering coefficient is a measure of the degree to which nodes in a graph tend to cluster together. Evidence suggests that in most real-world networks, and in particular social networks, nodes tend to create tightly knit groups characterised by a relatively high density of ties; this likelihood tends to be greater than the average probability of a tie randomly established between two nodes

Ref: https://en.wikipedia.org/wiki/Clustering_coefficient

In [ ]:
# Global Clustering Coefficient
print("Global Clustering Coefficient : " + str(np.mean(list(nx.clustering(G_karate).values()))))

# You can also calculate Clustering Coefficient for each community.
# For instance, identify the largest community ('giant' cluster) in a network.
# And then calculate the clustering coefficient for that giant cluster.

In [ ]:
# Returns shortest path between each node
nx.shortest_path(G_karate)

In [ ]:
# Returns shortest path length between each node
list(nx.all_pairs_shortest_path_length(G_karate))

# Find communities (Algorithm: Louvain)

In [ ]:
'''
You may need to install a new package "python-louvain"

pip install python-louvain
'''

############## Community Detection ##############
import community

In [ ]:
# Find communities (Algorithm: Louvain)
partition = community.best_partition(G_karate)  

In [ ]:
# Let's check out the communities
partition

In [ ]:
node_list = list(G_karate.nodes())

for node in node_list:
    print("Node ", node, " is in Cluster ", partition[node])

In [ ]:
# Let's graph the communities

pos = nx.spring_layout(G_karate)  # compute graph layout
plt.figure(figsize=(8, 8))  # image is 8 x 8 inches
plt.axis('off')

nx.draw_networkx_nodes(G_karate, pos, node_size=600, cmap=plt.cm.RdYlBu, node_color=list(partition.values()))
nx.draw_networkx_edges(G_karate, pos, alpha=0.3)

# plt.show(G_karate)

plt.show()

# Find communities (Algorithm: Girvan Newman)


In [ ]:
# Find communities(Algorithm: Girvan Newman)
from networkx.algorithms import community

k = 1
comp = community.girvan_newman(G_karate)

for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))

In [ ]:
# See if all communities are connected
nx.is_connected(G_karate)

# If the communities are NOT connected... how will you reach out to ALL nodes?
# Won't you have to find a starting node in each community & start spreading infection?

In [ ]:
# Suppose we have one community consisting of the following nodes:
community1 = [0, 1, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21]
# Note that, instead of hard-coding the nodes for community1
# You could programmatically retrieve the nodes from the Community Detection output


# Suppose we have a 2nd community consisting of:
community2 = [2, 8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
# Note that, instead of hard-coding the nodes for community2
# You could programmatically retrieve the nodes from the Community Detection output

In [ ]:
# We can extract out a sub-graph given a list of nodes

# Community 1
community1_subgraph = G_karate.subgraph(community1)

# Community 2
community2_subgraph = G_karate.subgraph(community2)

# Let's check out all the edges
print("============ Community 1 - Edges ===========")
print( community1_subgraph.edges() )

print("============ Community 2 - Edges ===========")
print( community2_subgraph.edges() )

# Find communities (Algorithm: Hierarchical Clustering)

In [ ]:
# AND here's yet another community detection algorithm --> Hierarchical Clustering

# Find communities(Algorithm: Hierarchical Clustering)
# Before applying hierachical clustering, we need to define the matrix of distances between each node.

# distances[i, j] is the length of the shortest path between i and j
pcc_longueurs=list(nx.all_pairs_shortest_path_length(G_karate))
distances=np.zeros((n,n))

for i in range(n):
    for j in range(n):
        distances[i, j] = pcc_longueurs[i][1][j]

# Perform clustering
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=2, linkage='average', affinity='precomputed').fit_predict(distances)

# Draw the graph
nx.draw(G_karate, node_color = clustering)

Below, we show you useful node-level centrality measures.

It is similar to closeness centrality, which is a measure of proximity.
High closeness centrality nodes can reach out to the rest of the network with the least number of links.
Hence, they can spread epidemics/rumors/information very fast throughout the network.

Eigenvector Centrality is a node-level influence measure.
A high eigenvector score means that a node is connected to many nodes who themselves have high scores.

Given two individuals Alice and Bob, each with 10 degree centrality value:
Alice could have a much higher eigenvector centrality than Bob because -
1) Each of Alice's 10 friends... has 10 friends
2) Each of Bob's 10 friends only has 1 friends each

In [ ]:
# Let's use the Karate Club network to calculate
# 1) Degree centrality
# 2) Closeness centrality
# 3) Betweenness centrality
# 4) Eigenvector centrality

c_degree = nx.degree_centrality(G_karate)
c_closeness = nx.closeness_centrality(G_karate)
c_betweenness = nx.betweenness_centrality(G_karate)
c_eigenvector = nx.eigenvector_centrality(G_karate)

c_degree = list(c_degree.values())
c_closeness = list(c_closeness.values())
c_betweenness = list(c_betweenness.values())
c_eigenvector = list(c_eigenvector.values())

In [ ]:
plt.figure(figsize=(18, 12))
f, axarr = plt.subplots(2, 2, num=1)
plt.sca(axarr[0,0])
nx.draw(G_karate, cmap = plt.get_cmap('inferno'), node_color = c_degree, node_size=300, pos=pos, with_labels=True)
axarr[0,0].set_title('Degree Centrality', size=16)

plt.sca(axarr[0,1])
nx.draw(G_karate, cmap = plt.get_cmap('inferno'), node_color = c_eigenvector, node_size=300, pos=pos, with_labels=True)
axarr[0,1].set_title('Eigenvalue Centrality', size=16)

plt.sca(axarr[1,0])
nx.draw(G_karate, cmap = plt.get_cmap('inferno'), node_color = c_closeness, node_size=300, pos=pos, with_labels=True)
axarr[1,0].set_title('Closeness Centrality', size=16)

plt.sca(axarr[1,1])
nx.draw(G_karate, cmap = plt.get_cmap('inferno'), node_color = c_betweenness, node_size=300, pos=pos, with_labels=True)
axarr[1,1].set_title('Betweenness Centrality', size=16)